In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import socket

# Function to find a free port
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.getsockname()[1]

# Load dataset
df = pd.read_csv('./df_ca2_dataviz.csv')





In [4]:
df.head()


,Year,Name,Sex,Count,IRE-Rank,USA-State-Total,Country,State
0,1985,Jessica,F,528,0,15645,USA,OR
1,1985,Jennifer,F,429,0,15645,USA,OR
2,1985,Amanda,F,428,0,15645,USA,OR
3,1985,Ashley,F,421,0,15645,USA,OR
4,1985,Sarah,F,356,0,15645,USA,OR


In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import socket

# Function to find a free port
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.getsockname()[1]

# Load dataset
df = pd.read_csv('./df_ca2_dataviz.csv')

# Initialize Dash app
app = dash.Dash(__name__)

# Function to convert input to sentence case
def to_sentence_case(name):
    return name.capitalize()

# App layout
app.layout = html.Div([
    html.Label('Enter a Name:'),
    dcc.Input(id='name-input', type='text', placeholder='Enter a name'),
    html.Br(),
    html.Div([
        dcc.Graph(id='heatmap'),
    ])
])

# Callback to update the heatmap based on user input
@app.callback(
    Output('heatmap', 'figure'),
    [Input('name-input', 'value')]
)
def update_heatmap(selected_name):
    if selected_name is None:
        selected_name = ''  # If no name is selected, use an empty string

    selected_name = to_sentence_case(selected_name)

    filtered_df = df[df['Name'] == selected_name]

    # Create heatmap trace
    heatmap_trace = go.Heatmap(
        z=filtered_df['Count'],
        x=filtered_df['State'],
        y=filtered_df['Country'],
        colorscale='Viridis',  # You can choose a different colorscale if needed
        colorbar=dict(title='Count'),
    )

    # Create layout
    layout = go.Layout(
        title=f'Count by State for {selected_name}',
        xaxis=dict(title='State'),
        yaxis=dict(title='Country'),
    )

    # Create figure
    fig = go.Figure(data=[heatmap_trace], layout=layout)

    return fig

# Run the app
if __name__ == '__main__':
    free_port = find_free_port()
    app.run_server(debug=True, port=free_port)
